In [1]:
library(raster)
library(gstat)
library(ggplot2)
library(gridExtra)
library(tmap)

library(RColorBrewer)



autofitVariogramPISCOp <- function (formula, input_data, model = c("Sph", "Exp", "Gau", 
                                                                   "Ste"), kappa = c(0.05, seq(0.2, 2, 0.1), 5, 10), fix.values = c(NA, 
                                                                                                                                    NA, NA), verbose = FALSE, GLS.model = NA, start_vals = c(NA, 
                                                                                                                                                                                             NA, NA), miscFitOptions = list(), boundaries, ...) 
{
  if ("alpha" %in% names(list(...))) 
    warning("Anisotropic variogram model fitting not supported, see the documentation of autofitVariogram for more details.")
  miscFitOptionsDefaults = list(merge.small.bins = TRUE, min.np.bin = 5)
  miscFitOptions = modifyList(miscFitOptionsDefaults, miscFitOptions)
  longlat = !is.projected(input_data)
  if (is.na(longlat)) 
    longlat = FALSE
  diagonal = spDists(t(bbox(input_data)), longlat = longlat)[1, 
                                                             2]
  if (!missing(boundaries)) 
    boundaries = boundaries
  else {
    boundaries = c(2, 4, 6, 9, 12, 15, 25, 35, 50, 65, 80, 
                   100) * diagonal * 0.35/100
  }
  if (!is(GLS.model, "variogramModel")) {
    experimental_variogram = variogram(formula, input_data, 
                                       boundaries = boundaries, ...)
  }
  else {
    if (verbose) 
      cat("Calculating GLS sample variogram\n")
    g = gstat(NULL, "bla", formula, input_data, model = GLS.model, 
              set = list(gls = 1))
    experimental_variogram = variogram(g, boundaries = boundaries, 
                                       ...)
  }
  if (miscFitOptions[["merge.small.bins"]]) {
    if (verbose) 
      cat("Checking if any bins have less than 5 points, merging bins when necessary...\n\n")
    while (TRUE) {
      if (length(experimental_variogram$np[experimental_variogram$np < 
                                           miscFitOptions[["min.np.bin"]]]) == 0 | length(boundaries) == 
          1) 
        break
      boundaries = boundaries[2:length(boundaries)]
      if (!is(GLS.model, "variogramModel")) {
        experimental_variogram = variogram(formula, 
                                           input_data, boundaries = boundaries, ...)
      }
      else {
        experimental_variogram = variogram(g, boundaries = boundaries, 
                                           ...)
      }
    }
  }
  if (is.na(start_vals[1])) {
    initial_nugget = min(experimental_variogram$gamma)
  }
  else {
    initial_nugget = start_vals[1]
  }
  if (is.na(start_vals[2])) {
    initial_range = 0.1 * diagonal
  }
  else {
    initial_range = start_vals[2]
  }
  if (is.na(start_vals[3])) {
    initial_sill = mean(c(max(experimental_variogram$gamma), 
                          median(experimental_variogram$gamma)))
  }
  else {
    initial_sill = start_vals[3]
  }
  if (!is.na(fix.values[1])) {
    fit_nugget = FALSE
    initial_nugget = fix.values[1]
  }
  else fit_nugget = TRUE
  if (!is.na(fix.values[2])) {
    fit_range = FALSE
    initial_range = fix.values[2]
  }
  else fit_range = TRUE
  if (!is.na(fix.values[3])) {
    fit_sill = FALSE
    initial_sill = fix.values[3]
  }
  else fit_sill = TRUE
  getModel = function(psill, model, range, kappa, nugget, 
                      fit_range, fit_sill, fit_nugget, verbose) {
    if (verbose) 
      debug.level = 1
    else debug.level = 0
    if (model == "Pow") {
      warning("Using the power model is at your own risk, read the docs of autofitVariogram for more details.")
      if (is.na(start_vals[1])) 
        nugget = 0
      if (is.na(start_vals[2])) 
        range = 1
      if (is.na(start_vals[3])) 
        sill = 1
    }
    obj = try(fit.variogram(experimental_variogram, model = vgm(psill = psill, 
                                                                model = model, range = range, nugget = nugget, kappa = kappa), 
                            fit.ranges = c(fit_range), fit.sills = c(fit_nugget, 
                                                                     fit_sill), debug.level = 0), TRUE)
    if ("try-error" %in% class(obj)) {
      warning("An error has occured during variogram fitting. Used:\n", 
              "\tnugget:\t", nugget, "\n\tmodel:\t", model, 
              "\n\tpsill:\t", psill, "\n\trange:\t", range, 
              "\n\tkappa:\t", ifelse(kappa == 0, NA, kappa), 
              "\n  as initial guess. This particular variogram fit is not taken into account. \nGstat error:\n", 
              obj)
      return(NULL)
    }
    else return(obj)
  }
  test_models = model
  SSerr_list = c()
  vgm_list = list()
  counter = 1
  for (m in test_models) {
    if (m != "Mat" && m != "Ste") {
      model_fit = getModel(initial_sill - initial_nugget, 
                           m, initial_range, kappa = 0, initial_nugget, 
                           fit_range, fit_sill, fit_nugget, verbose = verbose)
      if (!is.null(model_fit)) {
        vgm_list[[counter]] = model_fit
        SSerr_list = c(SSerr_list, attr(model_fit, "SSErr"))
      }
      counter = counter + 1
    }
    else {
      for (k in kappa) {
        model_fit = getModel(initial_sill - initial_nugget, 
                             m, initial_range, k, initial_nugget, fit_range, 
                             fit_sill, fit_nugget, verbose = verbose)
        if (!is.null(model_fit)) {
          vgm_list[[counter]] = model_fit
          SSerr_list = c(SSerr_list, attr(model_fit, 
                                          "SSErr"))
        }
        counter = counter + 1
      }
    }
  }
  strange_entries = sapply(vgm_list, function(v) any(c(v$psill, 
                                                       v$range) < 0) | is.null(v))
  if (any(strange_entries)) {
    if (verbose) {
      print(vgm_list[strange_entries])
      cat("^^^ ABOVE MODELS WERE REMOVED ^^^\n\n")
    }
    warning("Some models where removed for being either NULL or having a negative sill/range/nugget, \n\tset verbose == TRUE for more information")
    SSerr_list = SSerr_list[!strange_entries]
    vgm_list = vgm_list[!strange_entries]
  }
  if (verbose) {
    cat("Selected:\n")
    print(vgm_list[[which.min(SSerr_list)]])
    cat("\nTested models, best first:\n")
    tested = data.frame(`Tested models` = sapply(vgm_list, 
                                                 function(x) as.character(x[2, 1])), kappa = sapply(vgm_list, 
                                                                                                    function(x) as.character(x[2, 4])), SSerror = SSerr_list)
    tested = tested[order(tested$SSerror), ]
    print(tested)
  }
  result = list(exp_var = experimental_variogram, var_model = vgm_list[[which.min(SSerr_list)]], 
                sserr = min(SSerr_list))
  class(result) = c("autofitVariogram", "list")
  return(result)
}

                                                                                                    
                                                                                                    # obs_point_data: spatialpointsdataframe (columnas OBS y MODEL)
# model_grid_data: grid para interpolar

OK_interpolation <- function(obs_point_data,
                             model_grid_data,var=formula('mx~1'))
{
  
  # getting residuals
  
  # variogram autofit
  variogram_fit <- autofitVariogramPISCOp(var, data, model = c("Sph", "Exp", "Gau", "Ste")) 
  
  # ok
 
  gs <- gstat::gstat(formula = var, data = data, 
                    model = variogram_fit$var_model)
    
  kp <- raster::predict(gs, model_grid_data)
  round(raster::brick(kp)[["var1.pred"]], 3)
}

Warning message:
"package 'raster' was built under R version 3.6.3"Loading required package: sp
Warning message:
"package 'sp' was built under R version 3.6.3"Warning message:
"package 'gstat' was built under R version 3.6.3"Warning message:
"package 'ggplot2' was built under R version 3.6.3"Warning message:
"package 'gridExtra' was built under R version 3.6.3"Warning message:
"package 'tmap' was built under R version 3.6.3"

In [2]:
parameters = read.csv('D:/Proyectos_GitHub/Bartlet-Lewis_Regionalization/output/CV_parameters/parameters01.csv')
grd=readRDS('D:/Proyectos_GitHub/Bartlet-Lewis_Regionalization/data/grilla.rds')
data=parameters
coordinates(data) <- ~x+y
proj4string(data)=crs(grd)

In [3]:
formulas=c(formula('a~1'),formula('l~1'),formula('v~1'),formula('k~1'),formula('f~1'),formula('mx~1'))


for (k in 1:6){
    mapa=OK_interpolation(obs_point_data = data,model_grid_data = grd,var =formulas[[k]] )
    writeRaster(mapa,paste0('D:/Proyectos_GitHub/Bartlet-Lewis_Regionalization/output/CV_maps/','parametros','-',as.character(k),'.tif'),overwrite=TRUE)
}

Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"

[using ordinary kriging]


Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"

[using ordinary kriging]


Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"

...

[using ordinary kriging]


Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"

[using ordinary kriging]


Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"Warning message in wkt(obj):
"CRS object has no comment"

[using ordinary kriging]


In [48]:
forma=shapefile("~/biomaDatos/PISCO/peruShape/Limite Perú250_gcs.shp")
gauges=shapefile('~/R/estaciones/puntos.shp')

Mypal <- c('#ffffd9','#edf8b1','#c7e9b4','#7fcdbb','#41b6c4','#1d91c0'
           ,'#225ea8','#253494','#081d58')

grafico=function(mapa,nombre){
  resultado=
    tm_shape(mapa) +
    tm_raster(n=8,palette = Mypal,
              title=nombre)+
    tm_shape(gauges)+
    tm_dots('mx',legend.show = F,size=0.3,shape=4,col='black')+
    tm_legend(position = c("left", "bottom"))+
    tm_compass( position = c("left", "top"))+
    tm_grid(lwd = 0.2)+
    tm_shape(forma)+
    tm_borders("black",lwd = .5)
  resultado
}

a=expression(alpha)
l=expression(paste(lambda,'(1/hr)'))
v=expression(paste(upsilon,'(hr)'))
k=expression(kappa)
phi=expression(phi)
u=expression(paste(mu,'(mm/hr)'))


setwd('D:/Proyectos_GitHub/Bartlet-Lewis_Regionalization/output/CV_maps/')

mapa1=grafico(raster('parametros-1.tif'),nombre =a )
mapa2=grafico(raster('parametros-2.tif'),nombre = l)
mapa3=grafico(raster('parametros-3.tif'),nombre = v)
mapa4=grafico(raster('parametros-4.tif'),nombre = k)
mapa5=grafico(raster('parametros-5.tif'),nombre = phi)
mapa6=grafico(raster('parametros-6.tif'),nombre =u)

map_a=tmap_arrange(mapa1,mapa2,mapa3,mapa4,mapa5,mapa6,ncol = 3)

tmap_save(map_a, filename="D:/Proyectos_GitHub/Bartlet-Lewis_Regionalization/output/img/parameters.png")

Legend labels were too wide. The labels have been resized to 0.53, 0.53, 0.53, 0.53, 0.53, 0.53, 0.53. Increase legend.width (argument of tm_layout) to make the legend wider and therefore the labels larger.
Legend labels were too wide. The labels have been resized to 0.53, 0.53, 0.53, 0.53, 0.53, 0.53, 0.53. Increase legend.width (argument of tm_layout) to make the legend wider and therefore the labels larger.
Map saved to D:\Proyectos_GitHub\Bartlet-Lewis_Regionalization\output\img\parameters.png
Resolution: 2100 by 2100 pixels
Size: 7 by 7 inches (300 dpi)


In [49]:
setwd('D:/Proyectos_GitHub/Bartlet-Lewis_Regionalization/output/CV_maps/')
a=raster('parametros-1.tif')
v=raster('parametros-3.tif')
fi=raster('parametros-5.tif')
k=raster('parametros-4.tif')
l=raster('parametros-2.tif')
n=a/v

grafico=function(mapa,nombre){
  resultado=
    tm_shape(mapa) +
    tm_raster(n=8,palette = Mypal,
              title=nombre)+
    tm_shape(gauges)+
    tm_dots('mx',legend.show = F,size=0.3,shape=4,col='black')+
    tm_legend(position = c("left", "bottom"))+
    tm_compass( position = c("left", "top"))+
    tm_grid(lwd = 0.2)+
    tm_shape(forma)+
    tm_borders("black",lwd = .5)
  resultado
}

u=expression(paste(mu,'(mm/hr)'))

t1=grafico((a*k/v)^-1,expression(paste('mean hours between cells ',1/beta)))

t2=grafico(1/l, expression(paste('mean hours between storms',1/lambda)))

t3=grafico(n*fi,nombre =expression(paste('mean storm duration ',gamma,' (hr)')))

t4=grafico(n,nombre =expression(paste('cell duration ',eta,' (hr)')))

t5=grafico(raster('parametros-6.tif'),nombre =u)

map_b=tmap_arrange(t1,t2,t3,t4,t5,ncol = 3)



tmap_save(map_b, filename="D:/Proyectos_GitHub/Bartlet-Lewis_Regionalization/output/img/storm_characteristics.png")

Map saved to D:\Proyectos_GitHub\Bartlet-Lewis_Regionalization\output\img\storm_characteristics.png
Resolution: 2100 by 2100 pixels
Size: 7 by 7 inches (300 dpi)
